In [ ]:
RESOURCES_PATH = '../../resources'

In [ ]:
from pathlib import Path
from datetime import datetime
import re
import pandas as pd
import numpy as np

## Load dataset

In [ ]:
df = pd.read_csv(f'{RESOURCES_PATH}/dataset/original.csv')

print(f'Length: {len(df)}')
df.head()

In [ ]:
df = df[['object', 'financing', 'project', 'budget']]
df

In [ ]:
df.isnull().sum()

In [ ]:
df.drop(df[df.budget.isnull()].index, inplace=True)
df.isnull().sum()

In [ ]:
df.financing.replace('БЕЗ ВЦС', np.NaN, inplace=True)
df.isnull().sum()

In [ ]:
df.isnull().sum() / len(df)

In [ ]:
df.nunique()

## Replace year-specific target to current year target

In [ ]:
current_year = datetime.now().year - 2000

for i in range(10, current_year):
    df.budget.replace(f'Ппкс 20{i}', f'Ппкс 20{current_year}', inplace=True)
    df.budget.replace(f'Субсидия на ИЦ_ОЗОБ 20{i}', f'Субсидия на ИЦ_ОЗОБ 20{current_year}', inplace=True)

## Present the 'financing' feature as mask

In [ ]:
financing_mask_map = {
    'Ханьбань, КНР':     'Ханьбань, КНР',
    'Дальнефтепровод':   'Дальнефтепровод',
    'Фонд Оксфорда':     'Фонд Оксфорда',
    'ЭАЦ':               'ЭАЦ',
    'УНМ':               'УНМ',
    'Роснефть-классы':   'Роснефть-классы',
    'Д-000-00':          r'^Д-\d{1,3}-\d{2}(_.+)?(\/\d+)?$',
    '00-00-00000':       r'^\d{2}-\d{2,3}-\d{5}(\/\d{2})?(-П)?$',
    '000000/00000Д':     r'^\d{6,7}\/\d{4,5}Д$',
    '0000-0000/000-000': r'^\d{4}-\d{4}\/\d{3}-\d{3}$',
    '000-00Т':           r'^\d{3}\/\d{2}Т$',
    '0000-000-00-0':     r'^\d{4}-\d{3}-\d{2}-\d$',
    'DD-MM-YY':          r'^\d{2}-\d{2}-\d{2}$',
    '0/0000/0000':       r'^\d{1}\/\d{4}\/\d{4}$',
    '00.000.00.0000':    r'^\d{2}\.\d{3}\.\d{2}\.\d{4}$',
    '000-00-0000-000/0': r'^\d{3}-\d{2}-\d{4}-\d{3}\/\d$',
    'МК-0000.0000.0':    r'^МК-\d{4}\.\d{4}\.\d(\/\d{2})?$'
}

In [ ]:
def transform_financing(value):
    if str(value) == 'nan':
        return np.NaN

    for k in financing_mask_map:
        if re.match(financing_mask_map[k], value):
            return k

    return 'TO DELETE'

In [ ]:
df.financing = df.financing.apply(transform_financing)
df.financing.value_counts()

In [ ]:
df.drop(df[df.financing == 'TO DELETE'].index, inplace=True)

#### Define original dataset

In [ ]:
orig_df = df.copy()

## Clear to unique tuples

In [ ]:
df.fillna('None', inplace=True) # Fill N/A for correct grouping

In [ ]:
grouped_df = df.groupby(['object', 'financing', 'project', 'budget']).size().reset_index().rename(columns={0:'count'})
grouped_df

In [ ]:
unique_df = grouped_df.groupby(['object', 'financing', 'project'], as_index=False).apply(lambda x: x[x['count'] == x['count'].max()]).reset_index(drop=True)
unique_df

In [ ]:
print(f'Removed from original dataset (in current step): {round((1 - unique_df["count"].sum() / len(df)) * 100, 2)}%')

In [ ]:
unique_df.nunique()

## Clear too rare targets (these targets is obviously deprecated by business)

In [ ]:
budget_count_df = unique_df.groupby('budget').agg({'count': ['count', 'sum']})
budget_count_df.columns = ['count', 'original_count']
budget_count_df

In [ ]:
too_rare_budgets_df = budget_count_df[(budget_count_df['count'] < 3) | (budget_count_df['original_count'] < 150)]
too_rare_budgets_df

In [ ]:
budget_count_df[~budget_count_df.index.isin(too_rare_budgets_df.index)]

In [ ]:
cleared_df = unique_df[~unique_df.budget.isin(too_rare_budgets_df.index)]
cleared_df

In [ ]:
print(f'Removed from original dataset (in current step): {round((1 - cleared_df["count"].sum() / len(df)) * 100, 2)}%')

### Replace 'None' to NaN back

In [ ]:
orig_df.replace('None', np.NaN, inplace=True)
cleared_df.replace('None', np.NaN, inplace=True)

## Remove unknown categorical features from original dataset

In [ ]:
pd.DataFrame([cleared_df.nunique(), orig_df.nunique()])

In [ ]:
unknown_objects = set(orig_df.object.unique()).difference(set(cleared_df.object.unique()))
unknown_objects

In [ ]:
unknown_financing = set(orig_df.financing.unique()).difference(set(cleared_df.financing.unique()))
unknown_financing

In [ ]:
unknown_projects = set(orig_df.project.unique()).difference(set(cleared_df.project.unique()))
unknown_projects

In [ ]:
unknown_features = orig_df[orig_df.object.isin(unknown_objects) | orig_df.financing.isin(unknown_financing)| orig_df.project.isin(unknown_projects)].index
len(unknown_features)

In [ ]:
orig_df.drop(unknown_features, inplace=True)

In [ ]:
pd.DataFrame([cleared_df.nunique(), orig_df.nunique()])

## Save to TSV

In [ ]:
Path(f'{RESOURCES_PATH}/dataset/budget').mkdir(parents=True, exist_ok=True)

In [ ]:
orig_df.to_csv(f'{RESOURCES_PATH}/dataset/budget/original.tsv', index=False, sep='\t')
cleared_df.to_csv(f'{RESOURCES_PATH}/dataset/budget/cleared.tsv', index=False, sep='\t')

## List unique targets for label encoding

In [ ]:
cleared_targets = sorted(set(cleared_df.budget.unique()))
additional_orig_targets = sorted(set(orig_df.budget.unique()).difference(set(cleared_targets)))

with open(f'{RESOURCES_PATH}/dataset/budget/targets.txt', 'w') as fout:
    print(*cleared_targets, sep='\n', file=fout)
    print(*additional_orig_targets, sep='\n', file=fout)